In [13]:
import os
from langchain_community.utilities import SQLDatabase
from langchain_google_genai import GoogleGenerativeAI
from langchain_experimental.sql import SQLDatabaseChain

# Environment variables
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")

# Connection URI
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Initialize the database connection
db = SQLDatabase.from_uri(connection_uri)

# Update the import and instantiation
import google.generativeai as genai

# Configure the API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Create the LLM
llm = GoogleGenerativeAI(
    model="gemini-pro",  # Use the latest Gemini Pro model
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Create the SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

# Natural Language Query (NLQ)
nlq = "show loans above 500000"

# Generate SQL query
result = db_chain.run(nlq)
print(result)

PydanticUserError: `SQLDatabaseChain` is not fully defined; you should define `BaseCache`, then call `SQLDatabaseChain.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [10]:

import pydantic
import langchain
import google.generativeai

print("Pydantic version:", pydantic.__version__)
print("Langchain version:", langchain.__version__)
print("Google Generative AI version:", google.generativeai.__version__)

Pydantic version: 2.10.2
Langchain version: 0.3.8
Google Generative AI version: 0.8.3


In [11]:
import os
import sqlalchemy
from sqlalchemy import create_engine, text
import google.generativeai as genai

# Environment variables
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")

# Connection URI
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Create SQLAlchemy engine
engine = create_engine(connection_uri)

# Configure Google Generative AI
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Create Generative AI model
model = genai.GenerativeModel('gemini-pro')

# Function to generate SQL query
def generate_sql_query(natural_language_query):
    # Prompt to convert natural language to SQL
    prompt = f"""Convert the following natural language query to a SQL query:
    Natural Language Query: {natural_language_query}
    
    Please provide ONLY the SQL query, without any additional explanation."""

    # Generate the SQL query
    response = model.generate_content(prompt)
    return response.text.strip()

# Function to execute SQL query
def execute_query(sql_query):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            return result.fetchall()
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Natural Language Query
nlq = "show loans above 500000"

# Generate SQL query
try:
    sql_query = generate_sql_query(nlq)
    print("Generated SQL Query:", sql_query)

    # Execute the query
    results = execute_query(sql_query)
    
    # Print results
    if results:
        for row in results:
            print(row)
    else:
        print("No results found or query execution failed.")

except Exception as e:
    print(f"Error: {e}")

Generated SQL Query: ```sql
SELECT * FROM Loans WHERE Amount > 500000;
```
Error executing query: (pymssql.exceptions.ProgrammingError) (102, b"Incorrect syntax near '`'.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")
[SQL: ```sql
SELECT * FROM Loans WHERE Amount > 500000;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)
No results found or query execution failed.


In [15]:
import os
import sqlalchemy
from sqlalchemy import create_engine, text
import google.generativeai as genai

# Database Connection Setup
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Database Engine
engine = create_engine(connection_uri)

# Configure Google AI
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-pro')

# Function to get table schema
def get_table_schema():
    with engine.connect() as connection:
        # Retrieve table names
        table_query = """
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE'
        """
        tables = connection.execute(text(table_query)).fetchall()
        
        # Collect schema for each table
        table_schemas = {}
        for (table_name,) in tables:
            column_query = f"""
            SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = '{table_name}'
            """
            columns = connection.execute(text(column_query)).fetchall()
            table_schemas[table_name] = columns
        
        return table_schemas

# Generate SQL Query
def generate_sql_query(natural_language_query, table_schemas):
    # Prepare table schema information for the prompt
    schema_info = "Database Tables and Schemas:\n"
    for table, columns in table_schemas.items():
        schema_info += f"Table: {table}\n"
        schema_info += "Columns: " + ", ".join([f"{col[0]} ({col[1]})" for col in columns]) + "\n\n"

    # Comprehensive prompt with schema and query instructions
    prompt = f"""{schema_info}

Convert the following natural language query to a precise SQL query:
Natural Language Query: {natural_language_query}

Guidelines:
- Generate an accurate SQL query based on the table schemas
- Use appropriate JOIN if multiple tables are involved
- Handle filtering and conditions carefully
- Return ONLY the SQL query, without any explanation
"""

    # Generate SQL query
    response = model.generate_content(prompt)
    return response.text.strip()

# Execute Query
def execute_query(sql_query):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            return result.fetchall()
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Main Execution
def main():
    # Get table schemas
    table_schemas = get_table_schema()
    
    # Natural Language Query
    nlq = "show loans above 500000"
    
    # Generate and execute query
    try:
        sql_query = generate_sql_query(nlq, table_schemas)
        print("Generated SQL Query:", sql_query)

        results = execute_query(sql_query)
        
        if results:
            for row in results:
                print(row)
        else:
            print("No results found or query execution failed.")

    except Exception as e:
        print(f"Error: {e}")

# Run the script
main()

Generated SQL Query: ```sql
SELECT
  *
FROM LMS_Loan_Master
WHERE
  Application_LoanAmount > 500000;
```
Error executing query: (pymssql.exceptions.ProgrammingError) (102, b"Incorrect syntax near '`'.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\nDB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")
[SQL: ```sql
SELECT
  *
FROM LMS_Loan_Master
WHERE
  Application_LoanAmount > 500000;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)
No results found or query execution failed.


In [22]:
import os
import google.generativeai as genai

# Configure Google AI
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-pro')

def generate_query(query):
   # Generate SQL query
   sql_query = model.generate_content(
       f"Convert the following natural language query to SQL: {query}"
   ).text.strip()

   return sql_query

# Generate query
query = generate_query("show loans above 500000")


In [30]:
query.split('\n')[1]

'SELECT * FROM loans WHERE amount > 500000;'

In [32]:
import os
from langchain_community.utilities import SQLDatabase
from langchain_google_genai import GoogleGenerativeAI
from langchain_experimental.sql import SQLDatabaseChain

# Environment variables
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")

# Connection URI
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Initialize the database connection
db = SQLDatabase.from_uri(connection_uri)


In [33]:

# Create the LLM
llm = GoogleGenerativeAI(
   model="gemini-pro",
   google_api_key=os.getenv("GOOGLE_API_KEY")
)


In [34]:

# Create the SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=False)


PydanticUserError: `SQLDatabaseChain` is not fully defined; you should define `BaseCache`, then call `SQLDatabaseChain.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined

In [ ]:

# Natural Language Query (NLQ)
nlq = "show loans above 500000"

# Generate SQL query
result = db_chain.run(nlq)
print(result)

ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of

In [54]:
import os
from pydantic import BaseModel  # Add this import
from langchain_community.utilities import SQLDatabase
from langchain_google_genai import GoogleGenerativeAI
from langchain_experimental.sql import SQLDatabaseChain

# Define a BaseCache model
class BaseCache(BaseModel):
    pass  # You can add any specific attributes if needed

# Environment variables
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")

# Connection URI
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Initialize the database connection
db = SQLDatabase.from_uri(connection_uri, engine_args={"dialect": "mssql"})

# Create the LLM
llm = GoogleGenerativeAI(
   model="gemini-pro",
   google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Rebuild the model to resolve the definition issue
SQLDatabaseChain.model_rebuild()

# Create the SQLDatabaseChain
# db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=False)
db_chain = SQLDatabaseChain.from_llm(
    llm=llm, 
    db=db, 
    verbose=True,
    return_direct=True,
    sql_dialect="mssql"  # Specify MS SQL dialect
)

# Natural Language Query (NLQ)
nlq = "show loans above 500000"

try:
    result = db_chain.invoke(nlq)
except Exception as e:
    print(f"Query Generation Error: {e}")
    # Implement fallback or logging mechanism
# 

AttributeError: 'str' object has no attribute 'is_async'

In [49]:
sql_query = result['result'].split('\n')[1].split(':')[1].strip()

'Question: Show the latest 10 loans with loan amount greater than 1000000 and less than 2000000\nSQLQuery:SELECT TOP(10) [Loan_Id], [Application_No], [LoanAcNo], [Application_LoanAmount]\nFROM [LMS_Loan_Master]\nWHERE [Application_LoanAmount] > 1000000 and [Application_LoanAmount] < 2000000\nORDER BY [Loan_Id] DESC;'

In [55]:
from langchain_community.utilities import SQLDatabase
import sqlalchemy

# Recommended Connection Method
db = SQLDatabase.from_uri(
    connection_uri,
    engine_args={
        "connect_args": {
            "driver": "ODBC Driver 17 for SQL Server"
        }
    }
)


TypeError: connect() got an unexpected keyword argument 'driver'

In [17]:
import os
import google.generativeai as genai
from langchain_community.utilities import SQLDatabase
from langchain.chains.sql_database.prompt import SQL_PROMPTS

# Environment variables
SERVER = os.getenv("IP")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DATABASE = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD = os.getenv("ENCODED_PASSWORD")

# Connection URI
connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'

# Connect to the database
db = SQLDatabase.from_uri(connection_uri)
table_info = db.table_info

# Configure Google AI
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('models/text-bison-001')

# Define the custom prompt with SQL_PROMPTS["mssql"]
CUSTOM_PROMPT = (
    "You are an expert SQL developer. Convert the following natural language query "
    "into an optimized SQL query considering the provided table schema, and add some more relevent columns too.\n"
    "Table Schema:\n{table_info}\n"
    "Natural Language Query: {query}\n"
    "Optimized SQL Query:\n{sql_prompt}"
)

def generate_query(query):
    # Extract SQL prompt for MSSQL from LangChain
    sql_prompt = SQL_PROMPTS["mssql"]
    
    # Format the custom prompt with the table schema, user query, and MSSQL prompt
    prompt = CUSTOM_PROMPT.format(table_info=table_info, query=query, sql_prompt=sql_prompt)
    
    # Generate the SQL query using the formatted prompt
    sql_query = model.generate_content(prompt).text.strip()
    
    return sql_query

# Generate query
query = generate_query("show loans where loan amount is above 500000")
print(query)


SQLQuery:
```sql
SELECT
  *
FROM LMS_Loan_Master
WHERE
  Application_LoanAmount > 10500;
```


In [4]:
build agentic framework